## **LakeBeD-US Benchmark: High Frequency Data Wrangling**

## **Setup**

In [5]:
# IMPORT PACKAGES
import os
import pandas as pd

In [6]:
# SET WORKING DIRECTORY
os.chdir("/projects/ml4science/LakeBeD-US/LakeBeD-US-CSE-Benchmark/Data/1-Data-Wrangling")

In [8]:
acceptable_flags = [0, 5, 10, 19, 23, 25, 32, 43, 47, 51, 52]

## **Wrangling `ME_Mendota_2D.parquet`**

In [9]:
me_mendota = pd.read_parquet("ME_Mendota_2D.parquet")
me_mendota

variable,datetime,depth,flag,chla_rfu,do,fdom,par,phyco,temp
0,2006-06-28 02:31:00+00:00,0.0,0,NaN,NaN,NaN,NaN,NaN,22.26
1,2006-06-28 02:31:00+00:00,0.5,0,NaN,NaN,NaN,NaN,NaN,22.23
2,2006-06-28 02:31:00+00:00,1.0,0,NaN,NaN,NaN,NaN,NaN,22.28
3,2006-06-28 02:31:00+00:00,1.5,0,NaN,NaN,NaN,NaN,NaN,22.26
4,2006-06-28 02:31:00+00:00,2.0,0,NaN,NaN,NaN,NaN,NaN,22.26
...,...,...,...,...,...,...,...,...,...
101251364,2023-11-19 17:56:00+00:00,0.0,0,NaN,NaN,NaN,NaN,NaN,11.75
101251365,2023-11-19 17:57:00+00:00,0.0,0,NaN,NaN,NaN,NaN,NaN,11.84
101251366,2023-11-19 17:58:00+00:00,0.0,0,NaN,NaN,NaN,NaN,NaN,11.88
101251367,2023-11-19 17:59:00+00:00,0.0,0,NaN,NaN,NaN,NaN,NaN,11.88


### **Missing Values by Depth**

In [10]:
def compute_na_by_depth(data: pd.DataFrame) -> None:
	list_of_depths = list(data["depth"].unique())
	list_of_depths.sort()

	for depth in list_of_depths:
		if depth >= 0.0:
			print(f"Depth: {depth}")

			subset = data[(data["depth"] == depth) & (data["flag"].isin(acceptable_flags))]

			if len(subset) > 0:
				missing_percentages = (subset.isna().sum() / len(subset)) * 100
				print(missing_percentages)
			else:
				print("No data available for this depth.")
			
			print("")

In [11]:
compute_na_by_depth(me_mendota)

Depth: 0.0
variable
datetime      0.0
depth         0.0
flag          0.0
chla_rfu    100.0
do          100.0
fdom        100.0
par         100.0
phyco       100.0
temp          0.0
dtype: float64

Depth: 0.5
variable
datetime      0.0
depth         0.0
flag          0.0
chla_rfu    100.0
do          100.0
fdom        100.0
par         100.0
phyco       100.0
temp          0.0
dtype: float64

Depth: 1.0
variable
datetime     0.000000
depth        0.000000
flag         0.000000
chla_rfu     6.318595
do          19.197697
fdom        69.387670
par         51.304645
phyco        7.418325
temp         4.450048
dtype: float64

Depth: 1.5
variable
datetime      0.0
depth         0.0
flag          0.0
chla_rfu    100.0
do          100.0
fdom        100.0
par         100.0
phyco       100.0
temp          0.0
dtype: float64

Depth: 2.0
variable
datetime      0.0
depth         0.0
flag          0.0
chla_rfu    100.0
do          100.0
fdom        100.0
par         100.0
phyco       100.0
temp    

The most optimal depth to predict temperature and dissolved oxygen a, based on 
the number of missing values, is 1.0 meters from the surface.

### **Filtering**

In [12]:
me_mendota = me_mendota[(me_mendota["depth"] == 1.0) & (me_mendota["flag"].isin(acceptable_flags))]
me_mendota = me_mendota.drop(["depth", "flag"], axis = 1)
me_mendota

variable,datetime,chla_rfu,do,fdom,par,phyco,temp
2,2006-06-28 02:31:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
25,2006-06-28 02:32:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
48,2006-06-28 02:33:00+00:00,NaN,NaN,NaN,NaN,NaN,22.29
71,2006-06-28 02:34:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
94,2006-06-28 02:35:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
...,...,...,...,...,...,...,...
101251202,2023-11-19 15:22:00+00:00,1.38,10.03,7.26,198.3,0.46,9.04
101251205,2023-11-19 15:23:00+00:00,1.61,10.05,7.25,198.3,0.48,9.04
101251208,2023-11-19 15:24:00+00:00,1.61,10.05,7.25,244.4,0.48,9.04
101251211,2023-11-19 15:25:00+00:00,1.86,10.07,7.25,222.2,0.50,9.04


In [13]:
me_mendota_date_range = pd.DataFrame({"datetime": pd.date_range(me_mendota["datetime"].min(), me_mendota["datetime"].max(), freq = 'min')})
me_mendota = me_mendota.merge(me_mendota_date_range, on = "datetime", how = "outer")
me_mendota

,datetime,chla_rfu,do,fdom,par,phyco,temp
0,2006-06-28 02:31:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
1,2006-06-28 02:32:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
2,2006-06-28 02:33:00+00:00,NaN,NaN,NaN,NaN,NaN,22.29
3,2006-06-28 02:34:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
4,2006-06-28 02:35:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
...,...,...,...,...,...,...,...
9164201,2023-11-19 15:22:00+00:00,1.38,10.03,7.26,198.3,0.46,9.04
9164202,2023-11-19 15:23:00+00:00,1.61,10.05,7.25,198.3,0.48,9.04
9164203,2023-11-19 15:24:00+00:00,1.61,10.05,7.25,244.4,0.48,9.04
9164204,2023-11-19 15:25:00+00:00,1.86,10.07,7.25,222.2,0.50,9.04


### **Timescale Correction**

The time series is missing rows for certain dates; it skips a few dates at a 
time. We need to correct for this which will involve introducing missing 
values.

In [14]:
(me_mendota.isna().sum() / len(me_mendota)) * 100

datetime     0.000000
chla_rfu    49.714171
do          56.627350
fdom        83.568069
par         73.861554
phyco       50.304478
temp        48.711181
dtype: float64

Introducing the missing datetimes caused the missing value percentage to 
increase. However, we believe the unform distribution of observations on the 
entire timeline should not be problematic during imputation of the data after
the training-validation-testing split.

## **Wrangling `ME_NTL_HF_2D.parquet`**

In [22]:
me_ntl = pd.read_parquet("ME_NTL_HF_2D.parquet")
me_ntl

variable,datetime,depth,flag,temp
0,2018-12-01 06:00:00+00:00,1.0,0,3.582053
1,2018-12-01 06:00:00+00:00,1.5,0,3.603683
2,2018-12-01 06:00:00+00:00,2.0,0,3.675621
3,2018-12-01 06:00:00+00:00,2.5,0,3.612823
4,2018-12-01 06:00:00+00:00,3.0,0,3.624598
...,...,...,...,...
78617,2020-04-01 03:00:00+00:00,1.5,0,3.704339
78618,2020-04-01 03:00:00+00:00,2.0,0,3.776341
78619,2020-04-01 04:00:00+00:00,1.0,0,3.630451
78620,2020-04-01 04:00:00+00:00,1.5,0,3.660063


In [23]:
compute_na_by_depth(me_ntl)

Depth: 1.0
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64

Depth: 1.5
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64

Depth: 2.0
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64

Depth: 2.5
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64

Depth: 3.0
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64

Depth: 4.0
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64

Depth: 5.0
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64

Depth: 6.0
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64

Depth: 7.0
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64

Depth: 8.0
variable
datetime    0.0
depth       0.0
flag        0.0
temp        0.0
dtype: float64



## **Filtering**

In [24]:
me_ntl = me_ntl[(me_ntl["depth"] == 1.0) & (me_ntl["flag"].isin(acceptable_flags))]
me_ntl = me_ntl.drop(["depth", "flag"], axis = 1)
me_ntl

variable,datetime,temp
0,2018-12-01 06:00:00+00:00,3.582053
23,2018-12-01 07:00:00+00:00,3.589611
46,2018-12-01 08:00:00+00:00,3.617904
69,2018-12-01 09:00:00+00:00,3.686296
92,2018-12-01 10:00:00+00:00,3.713570
...,...,...
78607,2020-04-01 00:00:00+00:00,3.713415
78610,2020-04-01 01:00:00+00:00,3.710168
78613,2020-04-01 02:00:00+00:00,3.693512
78616,2020-04-01 03:00:00+00:00,3.675121


## **Timescale Correction**

In [25]:
me_ntl_date_range = pd.DataFrame({"datetime": pd.date_range(me_ntl["datetime"].min(), me_ntl["datetime"].max(), freq = 'min')})
me_ntl = me_ntl.merge(me_ntl_date_range, on = "datetime", how = "outer")
me_ntl

,datetime,temp
0,2018-12-01 06:00:00+00:00,3.582053
1,2018-12-01 06:01:00+00:00,NaN
2,2018-12-01 06:02:00+00:00,NaN
3,2018-12-01 06:03:00+00:00,NaN
4,2018-12-01 06:04:00+00:00,NaN
...,...,...
701156,2020-04-01 03:56:00+00:00,NaN
701157,2020-04-01 03:57:00+00:00,NaN
701158,2020-04-01 03:58:00+00:00,NaN
701159,2020-04-01 03:59:00+00:00,NaN


In [26]:
(me_ntl.isna().sum() / len(me_ntl)) * 100

datetime     0.000000
temp        99.113756
dtype: float64

## **Merge `ME_Mendota_2D.parquet` and `ME_NTL_HF_2D.parquet`**

In [27]:
mendota = me_mendota.merge(me_ntl, on = ["datetime", "temp"], how = "outer")
mendota

,datetime,chla_rfu,do,fdom,par,phyco,temp
0,2006-06-28 02:31:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
1,2006-06-28 02:32:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
2,2006-06-28 02:33:00+00:00,NaN,NaN,NaN,NaN,NaN,22.29
3,2006-06-28 02:34:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
4,2006-06-28 02:35:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
...,...,...,...,...,...,...,...
9457310,2023-11-19 15:22:00+00:00,1.38,10.03,7.26,198.3,0.46,9.04
9457311,2023-11-19 15:23:00+00:00,1.61,10.05,7.25,198.3,0.48,9.04
9457312,2023-11-19 15:24:00+00:00,1.61,10.05,7.25,244.4,0.48,9.04
9457313,2023-11-19 15:25:00+00:00,1.86,10.07,7.25,222.2,0.50,9.04


Let's make sure that the timescale doesn't timesteps after merging.

In [28]:
mendota_date_range = pd.DataFrame({"datetime": pd.date_range(mendota["datetime"].min(), mendota["datetime"].max(), freq = "min")})
mendota = mendota.merge(mendota_date_range, on = "datetime", how = "outer")
mendota

,datetime,chla_rfu,do,fdom,par,phyco,temp
0,2006-06-28 02:31:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
1,2006-06-28 02:32:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
2,2006-06-28 02:33:00+00:00,NaN,NaN,NaN,NaN,NaN,22.29
3,2006-06-28 02:34:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
4,2006-06-28 02:35:00+00:00,NaN,NaN,NaN,NaN,NaN,22.28
...,...,...,...,...,...,...,...
9457310,2023-11-19 15:22:00+00:00,1.38,10.03,7.26,198.3,0.46,9.04
9457311,2023-11-19 15:23:00+00:00,1.61,10.05,7.25,198.3,0.48,9.04
9457312,2023-11-19 15:24:00+00:00,1.61,10.05,7.25,244.4,0.48,9.04
9457313,2023-11-19 15:25:00+00:00,1.86,10.07,7.25,222.2,0.50,9.04


In [29]:
(mendota.isna().sum() / len(mendota)) * 100

datetime     0.000000
chla_rfu    51.272671
do          57.971591
fdom        84.077341
par         74.671659
phyco       51.844683
temp        50.235061
dtype: float64

In [30]:
mendota.to_parquet("./ME_HF_Cleaned.parquet")